In [1]:
import re
import pandas as pd 

In [56]:
f = open('whatsapp chat with shivani yadav.txt','r',encoding='utf-8')

In [3]:
data = f.read()

In [57]:
print (data)

In [5]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [6]:
messages = re.split(pattern, data)[1:]
messages

In [7]:
dates = re.findall(pattern, data)
dates

In [8]:
df = pd.DataFrame({'user_message':messages, 'message_date': dates})
# Convert message_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%y, %H:%M - ')

df.rename(columns={'message_date': 'date'}, inplace=True)

df.head()


In [9]:
df.shape

In [10]:
#separate user and message
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry [1:]: #user name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])
    
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()
    

In [11]:
df['year'] = df['date'].dt.year

In [12]:
df['month'] = df['date'].dt.month_name()

In [13]:
df['day'] = df['date'].dt.day

In [14]:
df['hour'] = df['date'].dt.hour

In [15]:
df['minute'] = df['date'].dt.minute

In [16]:
df.head()

In [17]:
df[df['user'] == 'Shivani Yadav'].shape


In [18]:
words = []
for message in df['message']:
    words.extend(message.split())

In [19]:
len (words)

In [20]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("let's www.gamil.com have URL stackoverflow.com as an example google.com, http://facebook.com, ftp://url.in .")
urls

In [21]:
links = []
for message in df ['message']:
    links.extend(extractor.find_urls(message))

In [22]:
len(links)

In [23]:
df

In [24]:
x = df['user'].value_counts().head()

In [25]:
import matplotlib.pyplot as plt

In [26]:
name = x.index
count = x.values

In [27]:
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [28]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index':'name','user':'percent'})

In [29]:
temp = df[df['user'] != 'group_notification']
temp = temp[temp['message'] != '<Media omitted>\n']

In [30]:
f = open('stop_hinglish (2).txt','r')
stop_words = f.read()
print(stop_words)

In [31]:
words = []

for message in temp['message']:
    for word in message.lower().split(): 
        if word not in stop_words:
            words.append(word)

In [32]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [33]:
import emoji 

In [34]:
emojis = []
for message in df ['message']:
    emojis.extend([c for c in message if c in emoji.EMOJI_DATA])

In [35]:
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))

In [36]:
df['month_num'] = df['date'].dt.month

In [37]:
timeline = df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [38]:
timeline

In [39]:
time = []
for i in range(timeline.shape[0]):
     time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))

In [40]:
timeline['time'] = time

In [41]:
timeline

In [42]:
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation='vertical')
plt.show()

In [43]:
df['only_date'] = df['date'].dt.date

In [44]:
daily_timeline = df.groupby('only_date').count()['message'].reset_index()

In [45]:
plt.figure(figsize=(18,10))
plt.plot(daily_timeline['only_date'],daily_timeline['message'])

In [46]:
df.head()

In [47]:
df['date'].dt.day_name()

In [48]:
df['day_name'] = df['date'].dt.day_name

In [49]:
df['day_name'].value_counts()

In [50]:
df['month'].value_counts()

In [51]:
period = []
for hour in df[['day_name','hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))
    
        


In [52]:
df['period'] = period

In [53]:
df.sample(5)

In [54]:
import seaborn as sns
plt.figure(figsize=(20,6))
sns.heatmap(df.pivot_table(index='day_name',columns='period' , values = 'message' , aggfunc = 'count').fillna(0))
plt.yticks(rotation = 'horizontal')
plt.show()

In [55]:
df.pivot_table(index='day_name',columns='period' , values = 'message' , aggfunc = 'count').fillna(0)